# Generate sysml-vocab.ttl

Generate the SysML v2 vocabulary from /Users/jamsden/Developer/SysML/SysML-v2-Pilot-Implementation/org.omg.sysml/model/SysML.ecore. This will be the merged KerML and SysML into a single vocabulary with a single namespace.



In [9]:
s = '<p>this is a test.</p>'
s[3:-4]

'this is a test.'

In [20]:
# Generate sysml-vocab.ttl
from rdflib import Graph, URIRef, Literal, Namespace, RDF, XSD, RDFS, OWL, DCTERMS
from pyecore.ecore import EClass, EAttribute, EReference, EString, EObject, EEnum
from pyecore.resources import ResourceSet, URI
from bs4 import BeautifulSoup
import os

# use the class name as a prefix to the attribute name to ensure they are unique
def qualident(eClass, eAttribute):
    return eClass.name[0].lower() + eClass.name[1:] + '_' + eAttribute.name[0].capitalize() + eAttribute.name[1:]  

# Get the description of the model element as the first paragraph
# with markdown removed (rdfs:comment is a string, not an XMLLiteral)
def comment(eModelElement, strip=True):
    desc = f'{eModelElement.name}.'  # rdfs:comment is required in the vocabulary, this is the default
    a = eModelElement.getEAnnotation('http://www.eclipse.org/emf/2002/GenModel')
    if a is None: return desc
    description = a.details['documentation']
    root = BeautifulSoup(str(description))
    description = root.find('p') 
    if strip:
        text =  description.get_text() # the first paragraph with the HTML stripped out
        if not (text is None or text==''): desc = text
    else:
        # strip off the <p> and </p>, they are not needed and cause the period to appear to be missing to shapechecker
        desc = str(description)[3:-4]  # but include the rest of the markup
    if not desc.endswith('.'): desc = desc + '.'
    return desc


# some useful RDF namespaces
vann = Namespace('http://purl.org/vocab/vann/')
oslc = Namespace('http://open-services.net/ns/core#')
oslc_am = Namespace('http://open-services.net/ns/am#')
oslc_sysmlv2 = Namespace('http://www.omg.org/spec/SysML/2.0#')

# create the SysML vocabulary graph, initially empty
g = Graph()
g.bind('oslc_sysmlv2', oslc_sysmlv2)
g.bind('oslc_am', oslc_am)

# add the vocabulary ontology
o = URIRef(oslc_sysmlv2)
g.add((o, RDF.type, OWL.Ontology))
g.add((o, RDFS.label, Literal("OSLC SysML v2 Vocabulary")))
g.add((o, DCTERMS.dateCopyrighted, Literal("2012-2024")))
g.add((o, DCTERMS.description, Literal("All vocabulary URIs defined in the OSLC SysML v2 namespace.", datatype=RDF.XMLLiteral)))
g.add((o, DCTERMS.hasVersion, Literal("PSD01")))
g.add((o, DCTERMS.isPartOf, URIRef('https://docs.oasis-open-projects.org/oslc-op/sysml/v2.0/os/sysml-spec.html')))
g.add((o, DCTERMS.issued, Literal("2024-04-25", datatype=XSD.date)))
g.add((o, DCTERMS.license, URIRef('http://www.apache.org/licenses/LICENSE-2.0')))
g.add((o, DCTERMS.publisher,  URIRef('https://open-services.net/about/')))
g.add((o, DCTERMS.source, URIRef('https://docs.oasis-open-projects.org/oslc-op/sysml/v2.0/os/sysml-vocab.ttl')))
g.add((o, DCTERMS.title, Literal("OSLC SysML v2 Vocabulary")))
g.add((o, vann.preferredNamespacePrefix, Literal("oslc_sysmlv2")))

# load the SysML.ecore model - this is the merged KerML and SysML metamodel using a single namespace
rset = ResourceSet()
resource = rset.get_resource(URI('/Users/jamsden/Developer/SysML/SysML-v2-Pilot-Implementation/org.omg.sysml/model/SysML.ecore'))
mm_root = resource.contents[0]
rset.metamodel_registry[mm_root.nsURI] = mm_root
# At this point, the .ecore is loaded in the 'rset' as a metamodel


for c in mm_root.eClassifiers:
    # Generate the classes
    if isinstance(c, EClass):
        g.add((oslc_sysmlv2.term(c.name), RDF.type, RDFS.Class))
        if c.name=='Element':
            g.add((oslc_sysmlv2.term(c.name), RDFS.subClassOf, oslc_am.Resource))  # only Element
        if c.eSuperTypes is not None and len(c.eSuperTypes) >= 1:
            for super in c.eSuperTypes:
                g.add((oslc_sysmlv2.term(c.name), RDFS.subClassOf, oslc_sysmlv2.term(super.name)))
        g.add((oslc_sysmlv2.term(c.name), RDFS.label, Literal(c.name)))
        g.add((oslc_sysmlv2.term(c.name), RDFS.comment, Literal(comment(c))))
        g.add((oslc_sysmlv2.term(c.name), RDFS.isDefinedBy, URIRef(oslc_sysmlv2)))

        # generate the properties
        for a in c.eStructuralFeatures:
            name = qualident(c, a)  # use the class name as a prefix to the attribute name to ensure they are unique
            s = oslc_sysmlv2.term(name)
            g.add((s, RDF.type, RDF.Property))
            g.add((s, RDFS.label, Literal(name)))
            g.add((s, RDFS.comment, Literal(comment(a))))
            g.add((s, RDFS.isDefinedBy, URIRef(oslc_sysmlv2)))
    if isinstance(c, EEnum):
        g.add((oslc_sysmlv2.term(c.name), RDF.type, RDFS.Class))
        g.add((oslc_sysmlv2.term(c.name), RDFS.label, Literal(c.name)))
        g.add((oslc_sysmlv2.term(c.name), RDFS.comment, Literal(comment(c))))
        g.add((oslc_sysmlv2.term(c.name), RDFS.isDefinedBy, URIRef(oslc_sysmlv2)))

        # generate the enumeration literals
        for a in c.eLiterals:
            name = qualident(c, a)  # use the class name as a prefix to the attribute name to ensure they are unique
            s = oslc_sysmlv2.term(name)
            g.add((s, RDF.type, oslc_sysmlv2.term(c.name)))
            g.add((s, RDFS.label, Literal(name)))
            g.add((s, RDFS.comment, Literal(comment(a))))
            g.add((s, RDFS.isDefinedBy, URIRef(oslc_sysmlv2)))
        
g.serialize(destination='../sysml-vocab.ttl')

# add the copywrite information
copywrite = """
# Copyright 2024 OASIS Open
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""
with open('../sysml-vocab.ttl','r') as f:
    with open('../temp.ttl','w') as f2: 
        f2.write(copywrite)
        f2.write(f.read())
os.remove('../sysml-vocab.ttl')
os.rename('../temp.ttl','../sysml-vocab.ttl')

# Generate sysml-shapes.ttl
Generate the SysML v2 constraints from /Users/jamsden/Developer/SysML/SysML-v2-Pilot-Implementation/org.omg.sysml/model/SysML.ecore. This will be the merged KerML and SysML into a single vocabulary with a single namespace. 

This depends on the sysml-vocab.ttl file that was generated above.



In [21]:
# Generate sysml-shapes.ttl

def multiplicity(eAttribute):
    lower = eAttribute.lowerBound
    upper = eAttribute.upperBound
    if lower==1 and upper==1: return oslc['Exactly-one']
    if lower==1 and upper !=0: return oslc['One-or-many']
    if lower==0 and upper==1: return oslc['Zero-or-one']
    if lower==0 and upper !=0: return oslc['Zero-or-many']

def valueType(a):
    match a.eType.name:
        case 'Boolean': return XSD.boolean
        case 'String': return XSD.string
        case 'Real': return XSD.float
        case 'Integer': return XSD.integer
        case _: return oslc_sysmlv2.term(a.eType.name)
        
    

# create the SysML constraints graph, initially empty
oslc_sysml_shapes = Namespace('https://www.omg.org/spec/SysML/shapes/20240801#')
jazz_am = Namespace('http://jazz.net/ns/dm/linktypes#')

g = Graph()
g.bind('oslc_sysmlv2', oslc_sysmlv2)
g.bind('oslc_am', oslc_am)
g.bind('oslc', oslc)
g.bind('', oslc_sysml_shapes)
g.bind('jazz_am', jazz_am)

s = URIRef(oslc_sysml_shapes)
g.add((s, RDF.type, oslc.ResourceShapeConstraints))
g.add((s, RDFS.label, Literal("OSLC  System Modeling Language (SysML) Constraints")))
g.add((s, DCTERMS.dateCopyrighted, Literal("2012-2024")))
g.add((s, DCTERMS.description, Literal("<p>All vocabulary URIs defined in the OSLC  System Modeling Language (SysML) namespace.</p>", datatype=RDF.XMLLiteral)))
g.add((s, DCTERMS.hasVersion, Literal("PSD01")))
g.add((s, DCTERMS.isPartOf, URIRef("https://docs.oasis-open-projects.org/oslc-op/sysml/v2.0/psd01/sysml-spec.html")))
g.add((s, DCTERMS.issued, Literal("2024-04-25", datatype=XSD.date)))
g.add((s, DCTERMS.license, URIRef("http://www.apache.org/licenses/LICENSE-2.0")))
g.add((s, DCTERMS.publisher, URIRef("https://open-services.net/about/")))
g.add((s, DCTERMS.source, URIRef("https://docs.oasis-open-projects.org/oslc-op/sysml/v2.0/psd01/sysml-shapes.ttl")))
g.add((s, DCTERMS.title, Literal("OSLC System Modeling Language (SysML) Version 2.0 Constraints")))

for c in mm_root.eClassifiers:
    # Generate the ResourceShape
    if isinstance(c, EClass):
        shape = oslc_sysml_shapes.term(c.name+'Shape')
        g.add((shape, RDF.type, oslc.ResourceShape))  # only Element
        g.add((shape, DCTERMS.title, Literal(c.name+'Shape', datatype=RDF.XMLLiteral)))
        g.add((shape, DCTERMS.description, Literal(comment(c,strip=False),datatype=RDF.XMLLiteral)))
        g.add((shape, oslc.describes, URIRef(oslc_sysmlv2.term(c.name))))

        # generate the properties to the ResourceShape
        for a in c.eStructuralFeatures:
            name = qualident(c, a)  # use the class name as a prefix to the attribute name to ensure they are unique
            g.add((shape, oslc.property, URIRef(oslc_sysml_shapes.term(name))))

            # and create the oslc:Property
            s = oslc_sysml_shapes.term(name)
            g.add((s, RDF.type, oslc.Property))
            g.add((s, oslc.name, Literal(name)))
            g.add((s, oslc.occurs, multiplicity(a)))
            g.add((s, oslc.propertyDefinition, URIRef(oslc_sysmlv2.term(name))))
            g.add((s, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
            if isinstance(a, EReference):
                g.add((s, oslc.valueType, oslc.Resource))
                g.add((s, oslc.range, oslc_sysmlv2.term(a.eType.name)))
                g.add((s, oslc.representation, oslc.Either))
            elif isinstance(a, EAttribute):
                g.add((s, oslc.range, valueType(a)))
            g.add((s, DCTERMS.description, Literal(comment(a,strip=False), datatype=RDF.XMLLiteral)))


        # add the inherited oslc_am:Resource properties
        g.add((shape, oslc.property, oslc_sysml_shapes.type))
        g.add((shape, oslc.property, oslc_sysml_shapes.dctype))
        g.add((shape, oslc.property, oslc_sysml_shapes.identifier))
        g.add((shape, oslc.property, oslc_sysml_shapes.title))
        g.add((shape, oslc.property, oslc_sysml_shapes.shortTitle))
        g.add((shape, oslc.property, oslc_sysml_shapes.description))
        g.add((shape, oslc.property, oslc_sysml_shapes.source))
        g.add((shape, oslc.property, oslc_sysml_shapes.creator))
        g.add((shape, oslc.property, oslc_sysml_shapes.created))
        g.add((shape, oslc.property, oslc_sysml_shapes.contributor))
        g.add((shape, oslc.property, oslc_sysml_shapes.modified))
        g.add((shape, oslc.property, oslc_sysml_shapes.serviceProvider))
        g.add((shape, oslc.property, oslc_sysml_shapes.instanceShape))
        g.add((shape, oslc.property, oslc_sysml_shapes.derives))
        g.add((shape, oslc.property, oslc_sysml_shapes.elaborates))
        g.add((shape, oslc.property, oslc_sysml_shapes.refine))
        g.add((shape, oslc.property, oslc_sysml_shapes.external))
        g.add((shape, oslc.property, oslc_sysml_shapes.satisfy))
        g.add((shape, oslc.property, oslc_sysml_shapes.trace))
        
    # and create the oslc_am:Resource inherited oslc:Property items
    g.add((oslc_sysml_shapes.type, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.type, oslc.name, Literal('type')))
    g.add((oslc_sysml_shapes.type, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.type, oslc.propertyDefinition, RDF.type))
    g.add((oslc_sysml_shapes.type, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.type, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.type, oslc.range, RDFS.Class))
    g.add((oslc_sysml_shapes.type, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.type, DCTERMS.description, Literal("The resource type URIs.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.dctype, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.dctype, oslc.name, Literal('dctype')))
    g.add((oslc_sysml_shapes.dctype, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.dctype, oslc.propertyDefinition, DCTERMS.type))
    g.add((oslc_sysml_shapes.dctype, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.dctype, oslc.valueType, XSD.string))
    g.add((oslc_sysml_shapes.dctype, DCTERMS.description, Literal("A short string representation for the type, for example ‘Car’.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.identifier, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.identifier, oslc.name, Literal('identifier')))
    g.add((oslc_sysml_shapes.identifier, oslc.occurs, oslc['Exactly-one']))
    g.add((oslc_sysml_shapes.identifier, oslc.propertyDefinition, DCTERMS.identifier))
    g.add((oslc_sysml_shapes.identifier, oslc.readOnly, Literal('true',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.identifier, oslc.valueType, XSD.string))
    g.add((oslc_sysml_shapes.identifier, DCTERMS.description, Literal("""A unique identifier for a resource. Typically read-only and assigned by the
service provider when a resource is created. Not typically intended for end-user display.""", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.title, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.title, oslc.name, Literal('title')))
    g.add((oslc_sysml_shapes.title, oslc.occurs, oslc['Exactly-one']))
    g.add((oslc_sysml_shapes.title, oslc.propertyDefinition, DCTERMS.title))
    g.add((oslc_sysml_shapes.title, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.title, oslc.valueType, RDF.XMLLiteral))
    g.add((oslc_sysml_shapes.title, DCTERMS.description, Literal("Title of the resource represented as rich text in XHTML content.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.shortTitle, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.shortTitle, oslc.name, Literal('shortTitle')))
    g.add((oslc_sysml_shapes.shortTitle, oslc.occurs, oslc['Zero-or-one']))
    g.add((oslc_sysml_shapes.shortTitle, oslc.propertyDefinition, oslc.shortTitle))
    g.add((oslc_sysml_shapes.shortTitle, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.shortTitle, oslc.valueType, RDF.XMLLiteral))
    g.add((oslc_sysml_shapes.shortTitle, DCTERMS.description, Literal("{{Short name identifying a resource, often used as an abbreviated identifier for presentation to end-users. SHOULD include only content that is valid inside an XHTML &lt;span&gt; element}}.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.description, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.description, oslc.name, Literal('description')))
    g.add((oslc_sysml_shapes.description, oslc.occurs, oslc['Zero-or-one']))
    g.add((oslc_sysml_shapes.description, oslc.propertyDefinition, DCTERMS.description))
    g.add((oslc_sysml_shapes.description, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.description, oslc.valueType, RDF.XMLLiteral))
    g.add((oslc_sysml_shapes.description, DCTERMS.description, Literal("Descriptive text about resource represented as rich text in XHTML content.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.source, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.source, oslc.name, Literal('source')))
    g.add((oslc_sysml_shapes.source, oslc.occurs, oslc['Zero-or-one']))
    g.add((oslc_sysml_shapes.source, oslc.propertyDefinition, DCTERMS.source))
    g.add((oslc_sysml_shapes.source, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.source, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.source, oslc.range, oslc.Any))
    g.add((oslc_sysml_shapes.source, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.source, DCTERMS.description, Literal("The resource URI a client can perform a get on to obtain the original non-OSLC AM formatted resource that was used to create this resource. The source resource is usually a binary or proprietary format that the service provider can consume and convert into an OSLC AM format. The service may use content negotiation with the Accept header to obtain the desired content type.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.creator, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.creator, oslc.name, Literal('creator')))
    g.add((oslc_sysml_shapes.creator, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.creator, oslc.propertyDefinition, DCTERMS.creator))
    g.add((oslc_sysml_shapes.creator, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.creator, oslc.valueType, oslc.AnyResource))
    g.add((oslc_sysml_shapes.creator, oslc.range, oslc.Any))
    g.add((oslc_sysml_shapes.creator, oslc.representation, oslc.Either))
    g.add((oslc_sysml_shapes.creator, DCTERMS.description, Literal("Creator or creators of the resource. It is likely that the target resource will be a foaf:Person but that is not necessarily the case.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.created, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.created, oslc.name, Literal('created')))
    g.add((oslc_sysml_shapes.created, oslc.occurs, oslc['Zero-or-one']))
    g.add((oslc_sysml_shapes.created, oslc.propertyDefinition, DCTERMS.created))
    g.add((oslc_sysml_shapes.created, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.created, oslc.valueType, XSD.dateTime))
    g.add((oslc_sysml_shapes.created, DCTERMS.description, Literal("Timestamp of resource creation.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.contributor, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.contributor, oslc.name, Literal('contributor')))
    g.add((oslc_sysml_shapes.contributor, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.contributor, oslc.propertyDefinition, DCTERMS.contributor))
    g.add((oslc_sysml_shapes.contributor, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.contributor, oslc.valueType, oslc.AnyResource))
    g.add((oslc_sysml_shapes.contributor, oslc.range, oslc.Any))
    g.add((oslc_sysml_shapes.contributor, oslc.representation, oslc.Either))
    g.add((oslc_sysml_shapes.contributor, DCTERMS.description, Literal("Contributor or contributors to the resource. It is likely that the target resource will be a foaf:Person but that is not necessarily the case.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.modified, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.modified, oslc.name, Literal('modified')))
    g.add((oslc_sysml_shapes.modified, oslc.occurs, oslc['Zero-or-one']))
    g.add((oslc_sysml_shapes.modified, oslc.propertyDefinition, DCTERMS.modified))
    g.add((oslc_sysml_shapes.modified, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.modified, oslc.valueType, XSD.dateTime))
    g.add((oslc_sysml_shapes.modified, DCTERMS.description, Literal("Timestamp of latest resource modification.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.serviceProvider, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.serviceProvider, oslc.name, Literal('serviceProvider')))
    g.add((oslc_sysml_shapes.serviceProvider, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.serviceProvider, oslc.propertyDefinition, oslc.serviceProvider))
    g.add((oslc_sysml_shapes.serviceProvider, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.serviceProvider, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.serviceProvider, oslc.range, oslc.ServiceProvider))
    g.add((oslc_sysml_shapes.serviceProvider, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.serviceProvider, DCTERMS.description, Literal("""A link to the resource's OSLC Service Provider. There may be cases when the
subject resource is available from a service provider that implements multiple domain
specifications, which could result in multiple values for this property.""", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.instanceShape, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.instanceShape, oslc.name, Literal('instanceShape')))
    g.add((oslc_sysml_shapes.instanceShape, oslc.occurs, oslc['Zero-or-one']))
    g.add((oslc_sysml_shapes.instanceShape, oslc.propertyDefinition, oslc.instanceShape))
    g.add((oslc_sysml_shapes.instanceShape, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.instanceShape, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.instanceShape, oslc.range, oslc.ResourceShape))
    g.add((oslc_sysml_shapes.instanceShape, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.instanceShape, DCTERMS.description, Literal("""The URI of a Resource Shape that describes the possible properties, occurrence,
value types, allowed values and labels. This shape information is useful in displaying the subject
resource as well as guiding clients in performing modifications. Instance shapes may be specific
to the authenticated user associated with the request that retrieved the resource, the current
state of the resource and other factors and thus should not be cached.""", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.derives, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.derives, oslc.name, Literal('derives')))
    g.add((oslc_sysml_shapes.derives, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.derives, oslc.propertyDefinition, jazz_am.derives))
    g.add((oslc_sysml_shapes.derives, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.derives, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.derives, oslc.range, oslc.Any))
    g.add((oslc_sysml_shapes.derives, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.derives, DCTERMS.description, Literal("""The resource that derives from another resource originated from or is
significantly influenced by the referenced resource. For example a model element derives from a
requirement.""", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.elaborates, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.elaborates, oslc.name, Literal('elaborates')))
    g.add((oslc_sysml_shapes.elaborates, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.elaborates, oslc.propertyDefinition, jazz_am.elaborates))
    g.add((oslc_sysml_shapes.elaborates, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.elaborates, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.elaborates, oslc.range, oslc.Any))
    g.add((oslc_sysml_shapes.elaborates, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.elaborates, DCTERMS.description, Literal("""This resource elaborates the referenced resource.""", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.refine, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.refine, oslc.name, Literal('refine')))
    g.add((oslc_sysml_shapes.refine, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.refine, oslc.propertyDefinition, jazz_am.refine))
    g.add((oslc_sysml_shapes.refine, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.refine, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.refine, oslc.range, oslc.Any))
    g.add((oslc_sysml_shapes.refine, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.refine, DCTERMS.description, Literal("""The target is a refinement of the source. (e.g. a use case scenario
might be a refinement of a textual requirement that describes the interaction).""", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.external, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.external, oslc.name, Literal('external')))
    g.add((oslc_sysml_shapes.external, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.external, oslc.propertyDefinition, jazz_am.external))
    g.add((oslc_sysml_shapes.external, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.external, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.external, oslc.range, oslc.Any))
    g.add((oslc_sysml_shapes.external, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.external, DCTERMS.description, Literal("""A generic link from a resource to an external web page.""", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.satisfy, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.satisfy, oslc.name, Literal('satisfy')))
    g.add((oslc_sysml_shapes.satisfy, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.satisfy, oslc.propertyDefinition, jazz_am.satisfy))
    g.add((oslc_sysml_shapes.satisfy, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.satisfy, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.satisfy, oslc.range, oslc.Any))
    g.add((oslc_sysml_shapes.satisfy, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.satisfy, DCTERMS.description, Literal("""The model element satisfies the requirement (e.g. The use case
satisfies a functional requirement).""", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.trace, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.trace, oslc.name, Literal('trace')))
    g.add((oslc_sysml_shapes.trace, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.trace, oslc.propertyDefinition, jazz_am.trace))
    g.add((oslc_sysml_shapes.trace, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.trace, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.trace, oslc.range, oslc.Any))
    g.add((oslc_sysml_shapes.trace, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.trace, DCTERMS.description, Literal("""The model element has a trace to the requirement (e.g. An attribute
or its value are traced to a requirement).""", datatype=RDF.XMLLiteral)))


g.serialize(destination='../sysml-shapes.ttl')

with open('../sysml-shapes.ttl','r') as f:
    with open('../temp.ttl','w') as f2: 
        f2.write(copywrite)
        f2.write(f.read())
os.remove('../sysml-shapes.ttl')
os.rename('../temp.ttl','../sysml-shapes.ttl')

## Get SysML v2 classes
A notebook to get a list of the SysML v2 classes and generate the vocabToShape div sections needed in sysml-shapes.html.



In [1]:
# Read and parse the sysml-vocab.ttl file
from rdflib import Graph, URIRef, Literal, Namespace, RDF

# some useful RDF namespaces
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace('http://www.w3.org/2000/01/rdf-schema#')
DCTERMS = Namespace('http://purl.org/dc/terms/')
oslc = Namespace('http://open-services.net/ns/core#')
oslc_am = Namespace('http://open-services.net/ns/am#')
oslc_sysmlv2 = Namespace('http://open-services.net/ns/sysmlv2#')

g = Graph()
g.parse('sysml-vocab.ttl')


<Graph identifier=Nd7d31ae2ce1f4e35b0d7eccca1f71236 (<class 'rdflib.graph.Graph'>)>

In [ ]:
# get a list of the classes
classes = g.subjects(rdf.type, rdfs.Class)

# and now create the shapeToSpec div for each class
for c in classes:
    name = str(c)
    name = name[name.index('#')+1:]
    print(
f"""

<div
        title="Constraints for {name}"
        data-include="./sysml-shapes.ttl#{name}Shape"
        data-oninclude="shapeToSpec"
        data-include-sync="true"
        data-include-replace="true"
        data-include-format="html"
      ></div>
"""        
    )